# Introduction

This notebook demonstrates the use of pre-trained MEGNet models to predict properties.

Author: Tsz Wai Ko (Kenko)
Email: t1ko@ucsd.edu

In [1]:
import torch
from pymatgen.core import Element, Structure, Lattice

# Import megnet related modules
from matgl.models.megnet import MEGNet

# MP Formation energy 

The pre-trained model is traned on the Materaisl Project mp.2018.6.1.json dataset.

In [2]:
# load the pre-trained MEGNet model. By default it is the formation energy model.
model = MEGNet.load("MP-2018.6.1-Eform")
# This is the structure obtained from the Materials Project.
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(4.1437), ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
eform = model.predict_structure(struct)
print(f"The predicted formation energy for CsCl is {float(eform.numpy()):.3f} eV/atom.")

The predicted formation energy for CsCl is -2.152 eV/atom.


/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  return th.as_tensor(data, dtype=dtype)
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/dgl/readout.py:443: DGLWarning: For a single graph, use a tensor of shape (1, *) for graph_feat. The support of shape (*) will be deprecated.
  dgl_warning(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped

# MP Band gap

The pre-trained model is traned on the Materails Project mp.2019.4.1.json dataset.

In [3]:
# Note that this is the multi-fidelity bandgap model.
model = MEGNet.load("MP-2019.4.1-BandGap-mfi")

# For mfi model, we need to define graph label ("0": PBE, "1": GLLB-SC, "2": HSE, "3": SCAN) 
graph_attrs = torch.tensor([0])
bandgap = model.predict_structure(struct, graph_attrs)
print(f"The predicted PBE BandGap for CsCl is {float(bandgap.numpy()):.3f} eV.")

The predicted PBE BandGap for CsCl is 5.094 eV.
